Hacer clouster sampling para modelo de maching learning.
Variables de bloqueo:
    * Mes
    * Dia
    * hora

In [12]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import time
import os
import re
import warnings # Para liminar warnings

Descargamos todos los set de datos

In [13]:
def ls(ruta = "./Datasets/taxi_trip"):
    return [arch.name for arch in os.scandir(ruta) if arch.is_file()]

In [14]:
archivos = ls()

In [15]:
archivos

['yellow_tripdata_2017-06.parquet',
 'yellow_tripdata_2017-07.parquet',
 'yellow_tripdata_2017-08.parquet',
 'yellow_tripdata_2017-09.parquet',
 'yellow_tripdata_2017-10.parquet',
 'yellow_tripdata_2017-11.parquet',
 'yellow_tripdata_2017-12.parquet',
 'yellow_tripdata_2018-01.parquet']

In [16]:
f = pd.read_parquet('datasets/output_2018-01.parquet')

In [20]:
def fxn():
    warnings.warn("deprecated", FutureWarning)

In [23]:
# indice en la muestra
collection = []
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
    
    # crear dataframe vacio para el sample
    muestra = pd.DataFrame()
    # Creamos las columnas del dataframe
    for i in f:
        muestra[i] = i

    # CODIGO PARA CREAR SAMPLES DE LOS N ARCHIVOS

    for file in tqdm (archivos, desc="Loading…", ascii=False, ncols=75):

        
        # --------------------CODIGO NECESARIO PARA CREAR EL SAMPLE---------------------------
        date = re.findall(r'\S*([0-9][0-9][0-9][0-9]-[0-9][0-9]).parquet', file)
        
        # Directorio para los archivos
        ruta = 'datasets/taxi_trip/yellow_tripdata_'+date[0]+'.parquet'
    #-----------------------------------------------------------------------------------------------------

        # Obtener cota de los meses
        final_date = date[0]+'-01'
        date = date[0].split('-')
        execution_date = datetime.datetime(int(date[0]), int(date[1]), 1)
        final_date = execution_date + datetime.timedelta(days = pd.Period(final_date).days_in_month )
    #-----------------------------------------------------------------------------------------------------
    
        # Cargamos dataset
        df = pd.read_parquet(ruta)
    #-----------------------------------------------------------------------------------------------------
    
        # eliminamos registros con fechas diferentes al mes en cuestion con base en la cota de los meses
        df.drop(df[df['tpep_pickup_datetime'] > final_date].index, inplace = True)
        df.drop(df[df['tpep_pickup_datetime'] < execution_date].index, inplace = True)
    #-----------------------------------------------------------------------------------------------------
    
        # creamos la mask para un dia y la hora
        mask_dia = np.zeros((df.tpep_pickup_datetime.shape[0], 1))
        mask_hora = np.zeros((df.tpep_pickup_datetime.shape[0], 1))
    
        for idx, i in enumerate(df.tpep_pickup_datetime):
            mask_dia[idx] =  i.strftime("%d")
        
        for idx, i in enumerate(df.tpep_pickup_datetime):
            mask_hora[idx] =  i.strftime("%H")
    #-----------------------------------------------------------------------------------------------------
    
        # Creamos los arrays en los que vamos a iterar
        dias = np.unique(mask_dia)
        horas = np.unique(mask_hora)
    #-----------------------------------------------------------------------------------------------------
    
        # Ajustamos las mascaras para poder usarlas
        mask_dia = mask_dia.sum(axis=1)
        mask_hora = mask_hora.sum(axis=1)
    #-----------------------------------------------------------------------------------------------------
    
        # Obtenemos la muestra
        for d in dias:
            for h in horas:    
                # dataframe filtrando por dia y hora
                aux = df[(mask_dia == d) & (mask_hora == h)]
        
                # obtenemos la muestra aleatoria del 12.5% de los datos
                mu = aux.sample(frac =.125)
        
                # Concatenamos mu con la muestra
                muestra = muestra.append(mu)
                del mu
                del aux

            #print('Dia '+str(d)+' finalizado')
    
    collection.append(muestra)
    del df
    del muestra
        

Loading…: 100%|███████████████████████████| 8/8 [2:03:58<00:00, 929.82s/it]


In [24]:
new = pd.concat(collection, axis=0)
new = new.reset_index()
new.drop(columns='index',inplace=True)
new.to_parquet('8_months_taxi.parquet')

In [65]:
muestra.to_csv('5_months_taxi.csv')

Pruebas que hice para construir la funcion

In [20]:
from taxi_trip import outliers_obt

In [ ]:
# indice en la muestra
collection = []
idx = 0

# DataFrame resultante
# columnas
col = ['date','trips','total_amount_sum','total_amount_mean','trip_distance_sum','trip_distance_mean']
muestra = pd.DataFrame()
for i in col:
    muestra[i] = i


with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
    # DataFrame resultante
    muestra = pd.DataFrame()
    for i in f:
        muestra[i] = i


    for file in tqdm (archivos, desc="Loading…", ascii=False, ncols=75):
        date = re.findall(r'\S*([0-9][0-9][0-9][0-9]-[0-9][0-9]).parquet', file)
        
        # Directorio para los archivos
        ruta = 'datasets/taxi_trip/yellow_tripdata_'+date[0]+'.parquet'
    #-----------------------------------------------------------------------------------------------------
        # Obtener cota de los meses
        final_date = date[0]+'-01'
        date = date[0].split('-')
        execution_date = datetime.datetime(int(date[0]), int(date[1]), 1)
        final_date = execution_date + datetime.timedelta(days = pd.Period(final_date).days_in_month - 1)
    #-----------------------------------------------------------------------------------------------------
    
        # Cargamos dataset
        df = pd.read_parquet(ruta)
    #-----------------------------------------------------------------------------------------------------
    
        # eliminamos registros con fechas diferentes a enero del 2018
        df.drop(df[df['tpep_pickup_datetime'] > final_date].index, inplace = True)
        df.drop(df[df['tpep_pickup_datetime'] < execution_date].index, inplace = True)
    #-----------------------------------------------------------------------------------------------------
    
    ## NORMALIZACION
    #('---------------------------------OUTLIERS---------------------------')
        df=outliers_obt(df,'total_amount','0.25','0.75',valoriqr=4.5)
        df=outliers_obt(df,'improvement_surcharge','0.25','0.75',valoriqr=4.5)
        df=outliers_obt(df,'tip_amount','0.25','0.75',valoriqr=2)
        df=outliers_obt(df,'mta_tax','0.25','0.75',valoriqr=1.5)
        df=outliers_obt(df,'extra','0.25','0.75',valoriqr=1.5)
        df=outliers_obt(df,'fare_amount','0.25','0.75',valoriqr=4.5)
        df=outliers_obt(df,'trip_distance','0.25','0.75',valoriqr=3)
        df=outliers_obt(df,'tpep_dropoff_datetime','0.25','0.75',valoriqr=1.5)
    #---------------------------------------------------------------------------------------------------

        # Drop columns
        df = df[['trip_distance','total_amount']].copy()
    
    #---------------------------------------------------------------------------------------------------

        # creamos la mask para un dia y la hora
        mask_dia = np.zeros((df.tpep_pickup_datetime.shape[0], 1))
        #mask_hora = np.zeros((df.tpep_pickup_datetime.shape[0], 1))
    
        for idx, i in enumerate(df.tpep_pickup_datetime):
            mask_dia[idx] =  i.strftime("%d")
        
        #for idx, i in enumerate(df.tpep_pickup_datetime):
        #    mask_hora[idx] =  i.strftime("%H")
    #-----------------------------------------------------------------------------------------------------
    
        # Creamos los arrays en los que vamos a iterar
        dias = np.unique(mask_dia)
        #horas = np.unique(mask_hora)
    #-----------------------------------------------------------------------------------------------------
    
        # Ajustamos las mascaras para poder usarlas
        mask_dia = mask_dia.sum(axis=1)
        #mask_hora = mask_hora.sum(axis=1)
    #-----------------------------------------------------------------------------------------------------
    
        # Obtenemos la muestra
        for d in tqdm (dias, desc="Loading…", ascii=False, ncols=75):
            # declaramos row
            row = []
            #for h in horas:    
            # dataframe filtrando por dia y hora
            #aux = df[mask_dia == d].copy()
            # dia
            fecha = datetime.datetime(int(date[0]), int(date[1]), d)
            
            # obtenemos las columnas
            row.append(fecha)
            row.append(df[mask_dia == d].shape[0])
            row.append(df[mask_dia == d].total_amount.sum())
            row.append(df[mask_dia == d].total_amount.median())
            row.append(df[mask_dia == d].trip_distance.sum())
            row.append(df[mask_dia == d].trip_distance.median())
    
            # Concatenamos mu con la muestra
            muestra.loc[idx] = row
            #print('Dia '+str(d)+' finalizado')
    
    collection.append(muestra)
        

In [22]:
muestra.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8821636 entries, 4145 to 8433068
Data columns (total 19 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   VendorID               object
 1   tpep_pickup_datetime   object
 2   tpep_dropoff_datetime  object
 3   passenger_count        object
 4   trip_distance          object
 5   RatecodeID             object
 6   store_and_fwd_flag     object
 7   PULocationID           object
 8   DOLocationID           object
 9   payment_type           object
 10  fare_amount            object
 11  extra                  object
 12  mta_tax                object
 13  tip_amount             object
 14  tolls_amount           object
 15  improvement_surcharge  object
 16  total_amount           object
 17  congestion_surcharge   object
 18  airport_fee            object
dtypes: object(19)
memory usage: 1.3+ GB


Cargamos los set de datos

In [30]:
# Cargamos dataset
df = pd.read_parquet('datasets/output_2018-01.parquet')

In [31]:
execution_date = datetime.datetime(2018, 1, 1)
final_date = datetime.datetime(2018, 1, 31)

In [32]:
# eliminamos registros con fechas diferentes a enero del 2018
df.drop(df[df['tpep_pickup_datetime'] > final_date].index, inplace = True)
df.drop(df[df['tpep_pickup_datetime'] < execution_date].index, inplace = True)

Dividir el dataframe por dia

In [33]:
# creamos la mask para un dia y la hora
mask_dia = np.zeros((df.tpep_pickup_datetime.shape[0], 1))
mask_hora = np.zeros((df.tpep_pickup_datetime.shape[0], 1))

In [34]:
for idx, i in enumerate(df.tpep_pickup_datetime):
    #print(i)
    #print(type(i.strftime("%d")))
    mask_dia[idx] =  i.strftime("%d")
    #print(mask_dia[idx])
    
for idx, i in enumerate(df.tpep_pickup_datetime):
    mask_hora[idx] =  i.strftime("%H")

In [35]:
# Creamos los arrays en los que vamos a iterar
dias = np.unique(mask_dia)
horas = np.unique(mask_hora)

In [46]:
# DataFrame resultante
muestra = pd.DataFrame()
for i in df:
    muestra[i] = i

In [37]:
# Ajustamos las mascaras para poder usarlas
mask_dia = mask_dia.sum(axis=1)
mask_hora = mask_hora.sum(axis=1)

In [38]:
df.tpep_pickup_datetime.shape

(8441287,)

In [ ]:
idx = 0
# Obtenemos la muestra
for d in tqdm (dias, desc="Loading…", ascii=False, ncols=75):
    for h in horas:    
        # dataframe filtrando por dia y hora
        aux = df[(mask_dia == d) & (mask_hora == h)].copy()

        # obtenemos la muestra aleatoria del 12.5% de los datos
        mu = aux.sample(frac =.125)

        # Concatenamos mu con la muestra
        muestra = muestra.append(mu)
        
    print(d)
    #print('Dia '+str(d)+' finalizado')


In [121]:
from tqdm import tqdm
import time


for i in tqdm (dias,
			desc="Loading…",
			ascii=False, ncols=75):
	#print(i)
	time.sleep(0.01)
	
print("Complete.")


Loading…: 100%|████████████████████████████| 31/31 [00:00<00:00, 63.26it/s]

Complete.


In [1]:
from multiprocessing import Process, Value, Array

def f(n, a):
    n.value = 3.1415927
    for i in range(len(a)):
        a[i] = -a[i]
        

if __name__ == '__main__':
    num = Value('d', 0.0)
    arr = Array('i', range(10))

    p = Process(target=f, args=(num, arr))
    p.start()
    p.join()

    print(num.value)
    print(arr[:])

0.0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
